In [1]:
import cv2
import dlib
import time


In [7]:
# Descargar herramientas de c++https://visualstudio.microsoft.com/es/visual-cpp-build-tools/
# Descargar el modelo desde: http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2

# Ruta al modelo preentrenado de predicción de 68 puntos de landmarks
predictor_path = "shape_predictor_68_face_landmarks.dat"

# Cargar el detector de rostros y el predictor de landmarks
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

In [22]:
import cv2
import dlib
from deepface import DeepFace
import numpy as np

# Path to the facial landmarks predictor model
predictor_path = "shape_predictor_68_face_landmarks.dat"

# Load face detector and landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)
video_capture = cv2.VideoCapture(0)

# Load Chopper's hat image with a transparent background
chopper_hat = cv2.imread("chopper_hat.png", cv2.IMREAD_UNCHANGED)  # Complete path to Chopper's hat

# Check if the image loaded correctly
if chopper_hat is None:
    print("Error: The image 'chopper_hat.png' could not be loaded. Check the file path.")
    video_capture.release()
    cv2.destroyAllWindows()
    exit(1)

def overlay_image(background, overlay, pos_x, pos_y):
    """
    Overlay an image with an alpha channel over another image at a specific position,
    ensuring it does not go out of bounds of the frame.
    """
    h, w = overlay.shape[:2]

    # Crop the overlay if it goes out of the frame bounds
    if pos_y < 0:
        overlay = overlay[-pos_y:, :]
        pos_y = 0
    if pos_x < 0:
        overlay = overlay[:, -pos_x:]
        pos_x = 0
    if pos_y + h > background.shape[0]:
        overlay = overlay[:background.shape[0] - pos_y, :]
    if pos_x + w > background.shape[1]:
        overlay = overlay[:, :background.shape[1] - pos_x]

    # Apply the overlay using the alpha channel
    alpha_overlay = overlay[:, :, 3] / 255.0  # Alpha channel for transparency
    for c in range(0, 3):
        background[pos_y:pos_y + overlay.shape[0], pos_x:pos_x + overlay.shape[1], c] = \
            (1 - alpha_overlay) * background[pos_y:pos_y + overlay.shape[0], pos_x:pos_x + overlay.shape[1], c] + \
            alpha_overlay * overlay[:, :, c]
    return background

while True:
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    try:
        faces = DeepFace.extract_faces(frame, detector_backend='yolov8')
        
        for face in faces:
            x, y = face['facial_area']['x'], face['facial_area']['y']
            w, h = face['facial_area']['w'], face['facial_area']['h']
            dlib_rect = dlib.rectangle(left=x, top=y, right=x + w, bottom=y + h)
            
            
            landmarks = predictor(gray, dlib_rect)
            
            # Get key points for eyes, temples, and chin for hat size and position
            left_eye = (landmarks.part(36).x, landmarks.part(36).y)
            right_eye = (landmarks.part(45).x, landmarks.part(45).y)
            left_temple = (landmarks.part(17).x, landmarks.part(17).y)  # Left temple point
            right_temple = (landmarks.part(26).x, landmarks.part(26).y)  # Right temple point
            chin = (landmarks.part(8).x, landmarks.part(8).y)

            # Calculate width and height for the hat
            head_width = int(2.5 * (right_temple[0] - left_temple[0]))  # Increased width for better coverage
            head_height = int(1.5 * abs(chin[1] - left_eye[1]))   # Adjusted height for appropriate look

            # Resize Chopper's hat
            resized_chopper_hat = cv2.resize(chopper_hat, (head_width, head_height))
            
            # Position the hat: centered over the forehead and head
            hat_pos_x = left_temple[0] - (head_width *2// 7)  # Center horizontally
            hat_pos_y = left_eye[1] - int(head_height * 0.95)  # Position higher to cover the top of the head

            # Overlay the hat on the image
            overlay_image(frame, resized_chopper_hat, hat_pos_x, hat_pos_y)
    except Exception as e:
        print(e)

    # Display the image with Chopper's hat overlay
    cv2.imshow("One Piece Filter", frame)
    if cv2.waitKey(5) & 0xFF == ord("q"):
        break

video_capture.release()
cv2.destroyAllWindows()


In [23]:
import cv2
import numpy as np
import torch
from deepface import DeepFace
from ultralytics import YOLO

# Set device to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load YOLOv8 model for face detection
yolo_model = YOLO("yolov8n.pt")  # Adjust model as needed

# Load images for each filter item
luffy_hat = cv2.imread("luffy_hat.png", cv2.IMREAD_UNCHANGED)
zoro_scar = cv2.imread("zoro_scar.png", cv2.IMREAD_UNCHANGED)
curly_eyebrow = cv2.imread("curly_eyebrow.png", cv2.IMREAD_UNCHANGED)
brook_afro = cv2.imread("brook_afro.png", cv2.IMREAD_UNCHANGED)
chopper_hat = cv2.imread("chopper_hat.png", cv2.IMREAD_UNCHANGED)

# Function to overlay image with rotation and scaling
def overlay_rotated_image(background, overlay, pos, angle=0, scale=1.0):
    h, w = overlay.shape[:2]
    overlay = cv2.resize(overlay, (int(w * scale), int(h * scale)))
    center = (overlay.shape[1] // 2, overlay.shape[0] // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_overlay = cv2.warpAffine(overlay, rotation_matrix, (overlay.shape[1], overlay.shape[0]), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)
    overlay_x, overlay_y = pos

    for y in range(rotated_overlay.shape[0]):
        for x in range(rotated_overlay.shape[1]):
            if overlay_y + y >= background.shape[0] or overlay_x + x >= background.shape[1]:
                continue
            alpha = rotated_overlay[y, x, 3] / 255.0
            background[overlay_y + y, overlay_x + x] = (1 - alpha) * background[overlay_y + y, overlay_x + x] + alpha * rotated_overlay[y, x, :3]
    return background

# Main loop
video_capture = cv2.VideoCapture(0)
while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    # Detect faces using YOLOv8
    results = yolo_model(frame)
    faces = []
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            faces.append((x1, y1, x2, y2))

    # Process each detected face
    for facial_area in faces:
        # Extract landmarks with DeepFace’s RetinaFace backend
        landmarks = None
        try:
            analysis = DeepFace.analyze(frame, actions=['emotion'], detector_backend="retinaface")
            landmarks = analysis[0]["region"]
        except Exception as e:
            print(f"Error detecting landmarks: {e}")

        if landmarks:
            # Position and scale Luffy's Hat
            left_temple = (landmarks["left_eye_x"], landmarks["left_eye_y"] - 20)
            right_temple = (landmarks["right_eye_x"], landmarks["right_eye_y"] - 20)
            hat_x = min(left_temple[0], right_temple[0]) - int(luffy_hat.shape[1] * 0.7)
            hat_y = left_temple[1] - luffy_hat.shape[0] // 2
            overlay_rotated_image(frame, luffy_hat, (hat_x, hat_y), scale=0.6)

            # Position and scale Zoro's Eye Scar
            scar_x = landmarks["left_eye_x"] - int(zoro_scar.shape[1] * 0.5)
            scar_y = landmarks["left_eye_y"]
            overlay_rotated_image(frame, zoro_scar, (scar_x, scar_y), scale=0.5)

            # Position and scale Sanji’s Curly Eyebrows
            eyebrow_scale = 0.6
            eyebrow_x_left = landmarks["left_eye_x"] - int(curly_eyebrow.shape[1] * eyebrow_scale // 2)
            eyebrow_y_left = landmarks["left_eye_y"] - 15
            eyebrow_x_right = landmarks["right_eye_x"] - int(curly_eyebrow.shape[1] * eyebrow_scale // 2)
            eyebrow_y_right = landmarks["right_eye_y"] - 15
            overlay_rotated_image(frame, curly_eyebrow, (eyebrow_x_left, eyebrow_y_left), scale=eyebrow_scale)
            overlay_rotated_image(frame, curly_eyebrow, (eyebrow_x_right, eyebrow_y_right), scale=eyebrow_scale)

            # Position and scale Brook’s Afro
            afro_width = int((right_temple[0] - left_temple[0]) * 1.5)
            afro_height = int(afro_width * brook_afro.shape[0] / brook_afro.shape[1])
            resized_afro = cv2.resize(brook_afro, (afro_width, afro_height))
            afro_x = left_temple[0] - afro_width // 4
            afro_y = left_temple[1] - afro_height // 2
            overlay_rotated_image(frame, resized_afro, (afro_x, afro_y))

            # Position and scale Chopper's Hat and Antlers
            hat_x_chopper = min(left_temple[0], right_temple[0]) - int(chopper_hat.shape[1] * 0.7)
            hat_y_chopper = left_temple[1] - chopper_hat.shape[0] // 2
            overlay_rotated_image(frame, chopper_hat, (hat_x_chopper, hat_y_chopper), scale=0.6)

    # Display the frame with filters
    cv2.imshow("One Piece Filters", frame)
    if cv2.waitKey(5) & 0xFF == ord("q"):
        break

video_capture.release()
cv2.destroyAllWindows()


Using device: cpu


FileNotFoundError: [Errno 2] No such file or directory: 'luffy_hat.png'